In [1]:
# Demo notebook for CCR and CVA

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from datetime import datetime
from typing import List

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from dnr.ccr.hw1f import HW1FModel
from dnr.ccr.fx_gbm import FXGBMModel
from dnr.ccr.market import MarketXCCYHW1F

from dnr.ccr.portfolio import Portfolio
from dnr.ccr.pricers.swap import Swap
from dnr.ccr.pricers.european_swaption import EuropeanSwaption
from dnr.ccr.pricers.yield_rate import YieldRate
from dnr.ccr.valuation import valuate
from dnr.ccr.valuation import extract_instance_df
from dnr.ccr.valuation import extract_scenario_df


In [4]:
# helper function



def plot_cube_scenarios(cube: pd.DataFrame, scenario_list: List[int]):
    for i in scenario_list:
        px.line(extract_scenario_df(cube, i), title=f"scenario: {i}").show()

def plot_cube_instances(cube: pd.DataFrame, instance_list: List[str]):
    for i in instance_list:
        px.line(extract_instance_df(cube, i), title=f"instance: {i}").show()

## Risk factors

For demo purposes, we simulate
* EUR interest rate curve: HW model
* USD interest rate curve: HW model
* EURUSD FX rate: GBM model

### Models

The Hull-White model is a one-factor model for the short rate $r(t)$:
$$
r(t) = x(t) + \phi(t)
$$
where
$$
dx(t) = - \alpha x(t) dt + \sigma dW(t) \quad \text{with} \quad x(0) = 0
$$

EURUSD FX rate is modeled as a geometric Brownian motion:
$$
dX(t)/X = (r^{\mathrm{USD}}(t) - r^{\mathrm{EUR}}(t)) dt + \sigma dW(t)
$$

In practice, dW's should be correlated, but for simplicity we assume they are independent.

### Setup risk factor simulation model

Consider the following risk factors:
* USD rate: $x^{\mathrm{USD}}(t)$
* EUR rate: $x^{\mathrm{EUR}}(t)$
* EURUSD FX: $X(t)$

In [5]:
# calibration date = today (simulation start date)
calib_dt = datetime(2024,4,15)

# interest rate model parameters and models
mr = 0.1
sigma = 0.01
phi_dates = [datetime(2024,7,15), datetime(2024,10,15), datetime(2025,1,15), datetime(2025,4,15)]
phi_values_eur = [0.03, 0.035, 0.04, 0.04]
phi_values_usd = [0.04, 0.045, 0.05, 0.05]

hw1f_eur = HW1FModel(calib_dt, mr, sigma, phi_dates, phi_values_eur)
hw1f_usd = HW1FModel(calib_dt, mr, sigma, phi_dates, phi_values_usd)

# FX GBM model
eurusd_fx = FXGBMModel(calib_dt, 1.2, 0.1)

### Risk factor simulation

Simulate the risk factors to generate a cube of 

* risk factors: USD rate, EUR rate, EURUSD FX
* time points: $t_1, t_2, \ldots, t_n$
* Monte Carlo scenarios: $s_1, s_2, \ldots, s_m$

For implementation simplicity, we use daily time points

In [6]:
# default values throughout this notebook
time_horizon = 11 * 365 # in days
save_freq = 7

In [7]:
# Pack the models into an object (Market) and run simulation over time
num_scen_small = 10
market = MarketXCCYHW1F("EUR", hw1f_eur, "USD", hw1f_usd, eurusd_fx, num_scen_small, np.random.default_rng(1234))
empty_portfolio = Portfolio([])
valuation_output = valuate("USD", empty_portfolio, market, time_horizon, save_freq)

### Risk factor cube

In [8]:
# risk factor simulation output
risk_factor_cube_df = valuation_output.risk_factor_cube_df
risk_factor_cube_df

EUR       USD    EURUSD
as_of_dt   scenario                              
2024-04-15 0         0.000000  0.000000  1.200000
           1         0.000000  0.000000  1.200000
           2         0.000000  0.000000  1.200000
           3         0.000000  0.000000  1.200000
           4         0.000000  0.000000  1.200000
...                       ...       ...       ...
2035-04-09 5         0.006416 -0.020775  0.729748
           6        -0.000764  0.006176  0.916139
           7         0.001058  0.014746  0.830147
           8        -0.018134 -0.000531  1.467536
           9        -0.006032 -0.041193  0.726269

[5740 rows x 3 columns]

Scenario view

In [9]:
# show the risk factor simulation for a given scenario
plot_cube_scenarios(risk_factor_cube_df, [0, 5])

risk factor view

In [10]:
plot_cube_instances(risk_factor_cube_df, ["USD", "EURUSD"])

## Derivation of simulated interest rate term-structure
The Hull-White model allows us to derivate an interest rate term-structure (**curves**) from a short rate.

$$
P(t, T) = e^{A(t, T) - B(t, T) r(t)}
$$

In [11]:
market = MarketXCCYHW1F("EUR", hw1f_eur, "USD", hw1f_usd, eurusd_fx, num_scen_small, np.random.default_rng(1234))
usd_yield_portfolio = Portfolio(
    [
        YieldRate(tnr, "USD", f"{int(tnr[:-1])*12}M") for tnr in ["1Y", "2Y", "5Y", "10Y", "20Y", "30Y"]
    ]
)
valuation_output = valuate("USD", usd_yield_portfolio, market, time_horizon, save_freq)

In [12]:
value_cube_df = valuation_output.value_cube_df
value_cube_df

1Y        2Y        5Y       10Y       20Y  \
as_of_dt   scenario                                                     
2024-04-15 0         0.046231  0.048066  0.048958  0.048784  0.047907   
           1         0.046231  0.048066  0.048958  0.048784  0.047907   
           2         0.046231  0.048066  0.048958  0.048784  0.047907   
           3         0.046231  0.048066  0.048958  0.048784  0.047907   
           4         0.046231  0.048066  0.048958  0.048784  0.047907   
...                       ...       ...       ...       ...       ...   
2035-04-09 5         0.030217  0.031116  0.033364  0.036031  0.039118   
           6         0.055861  0.055539  0.054567  0.053061  0.050764   
           7         0.064015  0.063305  0.061309  0.058476  0.054467   
           8         0.049479  0.049461  0.049290  0.048823  0.047865   
           9         0.010790  0.012612  0.017300  0.023129  0.030294   

                          30Y  
as_of_dt   scenario            
2024-04-15 0         0.047210  
           1         0.047210  
           2         0.047210  
           3         0.047210  
           4         0.047210  
...                       ...  
2035-04-09 5         0.040759  
           6         0.049290  
           7         0.052002  
           8         0.047167  
           9         0.034295  

[5740 rows x 6 columns]

In [13]:
plot_cube_scenarios(value_cube_df, [0, 5])

## Portfolio valuation

Create a portfolio of interest rate swaps and swaptions.

Using the simulated risk factors, revalue the portfolio for each scenario at each time point.

In [14]:
swap_usd10y =  Swap("USD 10Y",  "USD", datetime(2024,4,15), datetime(2034,4,15), "3M", "3M", 0.048, +1000000)
swap_usd5y =   Swap("USD 5Y",   "USD", datetime(2024,4,15), datetime(2029,4,15), "6M", "6M", 0.05, +1000000)
swap_eur10y =  Swap("EUR 10Y",  "EUR", datetime(2024,4,15), datetime(2034,4,15), "3M", "3M", 0.04, +1000000)
swap_usd2x10 = Swap("USD 2x10", "USD", datetime(2026,4,15), datetime(2036,4,15), "6M", "3M", 0.048, +1000000)
swaption_usd2x10 = EuropeanSwaption("USD Swaption 2x10", "USD", False, datetime(2026,4,15), 0.048, datetime(2036,4,15), +1000000)

demo_portfolio = Portfolio([swap_usd5y, swap_usd10y, swap_eur10y, swap_usd2x10, swaption_usd2x10])

### Illustration with a small number of scenarios

In [15]:
market = MarketXCCYHW1F("EUR", hw1f_eur, "USD", hw1f_usd, eurusd_fx, num_scen_small, np.random.default_rng(1234))
valuation_output = valuate("USD", demo_portfolio, market, time_horizon, save_freq)
value_cube_df = valuation_output.value_cube_df

In [16]:
value_cube_df

USD 5Y      USD 10Y       EUR 10Y      USD 2x10  \
as_of_dt   scenario                                                         
2024-04-15 0         3685.623228 -8412.618528  10255.546644 -10089.413674   
           1         3685.623228 -8412.618528  10255.546644 -10089.413674   
           2         3685.623228 -8412.618528  10255.546644 -10089.413674   
           3         3685.623228 -8412.618528  10255.546644 -10089.413674   
           4         3685.623228 -8412.618528  10255.546644 -10089.413674   
...                          ...          ...           ...           ...   
2035-04-09 5            0.000000     0.000000      0.000000  35165.493804   
           6            0.000000     0.000000      0.000000   1853.818130   
           7            0.000000     0.000000      0.000000  -8383.849534   
           8            0.000000     0.000000      0.000000   9747.270418   
           9            0.000000     0.000000      0.000000  58859.163935   

                     USD Swaption 2x10  
as_of_dt   scenario                     
2024-04-15 0              15537.710257  
           1              15537.710257  
           2              15537.710257  
           3              15537.710257  
           4              15537.710257  
...                                ...  
2035-04-09 5              35165.493804  
           6               1853.818130  
           7                 -0.000000  
           8               9747.270418  
           9              58859.163935  

[5740 rows x 5 columns]

In [17]:
plot_cube_scenarios(value_cube_df, [0, 5])

In [18]:
plot_cube_instances(value_cube_df, ["USD 2x10", "USD Swaption 2x10"])

### Illustration with a large number of scenarios

For risk measure calculations, we need a large number of scenarios. We will use 3000 scenarios.

In [19]:
# other portfolios to use as examples

demo_portfolio2 = Portfolio([
    Swap("USD 10Y",  "USD", datetime(2024,4,15), datetime(2034,4,15), "3M", "3M", 0.048, +1000000),
    Swap("USD 7Y",   "USD", datetime(2024,4,15), datetime(2031,4,15), "6M", "6M", 0.05, -1000000)
])

demo_portfolio3 = Portfolio([
    Swap("USD 2x10", "USD", datetime(2026,4,15), datetime(2036,4,15), "6M", "3M", 0.048, +1000000),
    EuropeanSwaption("USD Swaption 2x10", "USD", False, datetime(2026,4,15), 0.048, datetime(2036,4,15), -1000000)
])

In [20]:
#my_portfolio = demo_portfolio
#my_portfolio = demo_portfolio2
my_portfolio = demo_portfolio3

In [21]:
num_scen_large = 3000
market = MarketXCCYHW1F("EUR", hw1f_eur, "USD", hw1f_usd, eurusd_fx, num_scen_large, np.random.default_rng(1234))
valuation_output = valuate("USD", my_portfolio, market, time_horizon, save_freq)
value_cube_df = valuation_output.value_cube_df
value_cube_df

USD 2x10  USD Swaption 2x10
as_of_dt   scenario                                 
2024-04-15 0        -10089.413674      -15537.710257
           1        -10089.413674      -15537.710257
           2        -10089.413674      -15537.710257
           3        -10089.413674      -15537.710257
           4        -10089.413674      -15537.710257
...                           ...                ...
2035-04-09 2995       1544.417881       -1544.417881
           2996      24681.258568          -0.000000
           2997       7835.265310       -7835.265310
           2998      30159.562442          -0.000000
           2999      -7181.334410           0.000000

[1722000 rows x 2 columns]

In [22]:
plot_cube_scenarios(value_cube_df, [0, 2999])

## Risk Measures

* EE (or EPE): average of (positive) exposures
* PFE: 95% quantile of exposures

Both are calculated for each time point and form a curve.

We can apply them to 
* Each trade in the portfolio.
* Aggregated over all trades, either gross or netted

###  Trade level exposures

In [23]:
trade_exposure_df = pd.concat({
        "EE": np.maximum(value_cube_df, 0.0).groupby(level=0, axis=0).mean(),
        "PFE": np.maximum(value_cube_df, 0.0).groupby(level=0, axis=0).quantile(0.95)
}, axis=0)


In [24]:
for exp_type in ['EE', 'PFE']:
    px.line(trade_exposure_df.loc[exp_type], title=exp_type).show()

In [25]:
for trade in my_portfolio.get_trade_ids():
    px.line(trade_exposure_df.loc[:, trade].unstack(0), title=trade).show()

### Portfolio level exposures

In [26]:
portfolio_exposure_df = pd.concat(
    {
        "Gross EE": np.maximum(value_cube_df, 0.0).sum(axis=1).groupby(level=0).mean(),
        "Gross PFE": np.maximum(value_cube_df, 0.0).sum(axis=1).groupby(level=0).quantile(0.95),
        "Net EE": np.maximum(value_cube_df.sum(axis=1), 0.0).groupby(level=0).mean(),
        "Net PFE": np.maximum(value_cube_df.sum(axis=1), 0.0).groupby(level=0).quantile(0.95)        
    }
).unstack(0)

In [27]:
px.line(portfolio_exposure_df).show()

# CVA calculation

Let's perform on the netted exposure.

Assume the implied PD is constant over time. In practice, PD has a term structure.

In [48]:
ee_ds = portfolio_exposure_df['Net EE']
pd_per_annum = 0.01 # 1% per annum
recovery_rate = 0.4
time_horizon_years = (ee_ds.index - calib_dt)/np.timedelta64(365, 'D')

DF = hw1f_usd.get_discount_factor(0, time_horizon_years, 0.0)

expected_default_ds = DF[1:] * ee_ds.iloc[1:] * pd_per_annum * (time_horizon_years[1:] - time_horizon_years[:-1])

318.3163064596638

In [56]:
px.line(expected_default_ds, markers=True).show()

In [57]:
CVA = (1-recovery_rate) * expected_default_ds.sum()
CVA

318.3163064596638